In [1]:
import pandas as pd
import json

# Cargar productos desde products_utf8.csv
productos_csv = pd.read_csv('../data/raw/products_utf8.csv', usecols=[
    "product_name", "energy_per_100", "fat_per_100", "saturates_per_100",
    "salt_per_100", "sugar_per_100", "carbohydrate_per_100", "protein_per_100",
    "fibre_per_100", "serving_value", "serving_unit", "ingredients_text"
], encoding='utf-8')

# Reemplazar NaN por cadenas vacías en ingredients_text
productos_csv['ingredients_text'] = productos_csv['ingredients_text'].fillna('')

# Cargar los datos de ingredientes identificados desde el JSON
with open("productos_ingredientes_identificados_v24.json", "r", encoding="utf-8") as file:
    productos_ingredientes = json.load(file)

productos_ingredientes_df = pd.DataFrame(productos_ingredientes)

# Unir ambos datasets usando "ingredients_text" como clave
productos_combinados = productos_csv.merge(
    productos_ingredientes_df[["ingredients_text", "ingredientes_identificados", "ingredientes_con_porcentaje"]],
    on="ingredients_text",
    how="left"
)

# Cargar los ingredientes normalizados
ingredientes_totales = pd.read_csv("ingredientes+huella.csv", usecols=["ingredient_normalized"])
ingredientes_totales_list = ingredientes_totales["ingredient_normalized"].tolist()

# Inicializar columnas de ingredientes normalizados con 0
for ingrediente in ingredientes_totales_list:
    productos_combinados[ingrediente] = 0

# Asignar valores 0 o 1 según los ingredientes identificados
for index, row in productos_combinados.iterrows():
    if isinstance(row["ingredientes_identificados"], list):
        for ingrediente in row["ingredientes_identificados"]:
            if ingrediente in ingredientes_totales_list:
                productos_combinados.at[index, ingrediente] = 1

# Añadir una columna para porcentaje y dominante si existen
productos_combinados["ingrediente_dominante"] = productos_combinados["ingredientes_con_porcentaje"].apply(
    lambda x: x[0]["ingrediente"] if isinstance(x, list) and len(x) > 0 else None
)
productos_combinados["porcentaje_dominante"] = productos_combinados["ingredientes_con_porcentaje"].apply(
    lambda x: x[0]["porcentaje"] if isinstance(x, list) and len(x) > 0 else None
)

# Seleccionar columnas finales
columnas_finales = ["product_name", "energy_per_100", "fat_per_100", "saturates_per_100",
                    "salt_per_100", "sugar_per_100", "carbohydrate_per_100", "protein_per_100",
                    "fibre_per_100", "serving_value", "serving_unit"] + ingredientes_totales_list + [
                    "ingrediente_dominante", "porcentaje_dominante"]

dataset_final = productos_combinados[columnas_finales]

# Exportar el dataset final a un archivo CSV
dataset_final.to_csv("dataset_final.csv", index=False, encoding="utf-8")

print("El dataset final se ha generado y guardado como 'dataset_final.csv'.")


C:\Users\Usuario\AppData\Local\Temp\ipykernel_3300\1068248572.py:5: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_csv = pd.read_csv('../data/raw/products_utf8.csv', usecols=[


MemoryError: Unable to allocate 35.4 GiB for an array with shape (10, 474873178) and data type object